Устанавливаем библиотеки: pandas - для работы с таблицами, pyreadstat - для чтения файлов из статистических программ (типа SPSS). Обязательно указываем версию библиотеки через ==, потому что код, работающий в одной версии, может не работать в другой.  
Документации на них:

https://pandas.pydata.org

https://ofajardo.github.io/pyreadstat_documentation/_build/html/index.html


In [1]:
!pip install pandas==1.4.2 pyreadstat==1.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 4.2 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.4.4
    Uninstalling pandas-1.4.4:
      Successfully uninstalled pandas-1.4.4


Подключаем Google Disk, надо будет дать разрешение этой jupyter-notebook тетрадке на чтение/запись с него. Файлы *.sav должны быть заранее загружены на Google Disk. Если они загружены не в корень диска, а в подпапку, надо будет поменять переменные fpath (cм. ниже)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# импортируем библиотеку pyreadstat, 
# которая ориентирована на работу с форматами статистических (в т.ч., платных) программ.
import pyreadstat
import numpy as np

In [5]:
fpath_2016 = 'drive/MyDrive/коуж/IND_2016.sav'
df_2016, meta_2016 = pyreadstat.read_file_multiprocessing(pyreadstat.read_sav, fpath_2016, num_processes=16, encoding='cp1251') 

In [6]:
fpath_2018 = 'drive/MyDrive/коуж/IND_2018.sav'
df_2018, meta_2018 = pyreadstat.read_file_multiprocessing(pyreadstat.read_sav, fpath_2018, num_processes=16, encoding='cp1251') 

In [8]:
# В программе учебной дисциплины заявлен анализ по 2011, 2014 году, но в данных по ним нет нужных нам переменных.

# Источник данных: https://gks.ru/ .

fpath_2020 = 'drive/MyDrive/коуж/IND_OSN_2020.sav'
df_2020, meta_2020 = pyreadstat.read_file_multiprocessing(pyreadstat.read_sav, fpath_2020, num_processes=16, encoding='cp1251') 
selection = [
     'I09_09', # Оценка безопасности района проживания респондента в темное время суток',    
     'I05_39', # Предпочтительное время работы',
     "H00_02", # код субъекта РФ
    ] 

In [9]:
def descriptional_statistics(df, meta, selection):
    # Создадим новую таблицу (pandas DataFrame), но только с теми колонками,
    # которые используются в расчётах.
    # Так используется меньше памяти, удобнее визуализировать. Но можно пользоваться и исходной таблицей.
    selected = df[selection]
    # Используем генератор списков, чтобы переименовать служебные заголвки в читаемые.
    # для этого мы хотим "пройтись" по всем меткам в списке selection и получить новый список вместо selection.
    # Подробнее о генераторах списков: https://pythontutor.ru/lessons/lists/#section_3
    selection_names = [meta.column_names_to_labels[label] for label in selection]
    # Заменим исходные наименования колонок на наименования из получившегося списка selection_names, 
    # теперь у нас наименования колонок будет как в столбце "метки" файла sav.
    selected.columns = selection_names
    # 98 - код Республики Саха (Якутии), согласно ОКВЭДам.
    # создаем маску, чтобы получить таблицу с данными только по Якутии.
    mask = selected['Код субъекта РФ'] == '98'
    # применяем маску - используем только те строчки, где значения маски True
    region = selected[mask]
    # число наблюдений в таблице получаем через атрибут shape.
    # так как в shape находится объект типа кортеж (tuple)- структура данных, похожая на список,
    # но неизменяемая (чтобы добавить или удалить элемент, придется создавать новый объект типа кортеж в памяти).
    # из кортежа,  так же, как из списка, можно получить по порядковому индексу его элемент
    # в данном случае у нас shape[0] - количество строк, shape[1] - количество столбцов.
    # pandas DataFrame - двумерная таблица, ее размерность описывается количеством строчек и столбцов.
    # одномерная "таблица" pandas, по сути, одна колонка, размерность которой описывается только количеством строчек,
    # называется Series

    print(f"Количество наблюдений по выбранным переменным: {region.shape[0]}")
    # apply - метод pandas DataFrame, который позволяет применить функцию ко всему Series или DataFrame,
    # не прописывая цикл явно.

    # Применим функцию, который переведет служебные значения в читаемые ко всему столбцу Размер населенного пункта.

    # в python есть два способа объявить функцию - через ключевое слово def или lambda.
    # def удобно использовать, когда у функции много аргументов и/или в ней выполняется много действий
    # lambda удобно использовать, когда функция принимает 1-2 аргумента и выполняемые действия можно прописать в 
    # одну строчку

    # После ключевого слова lambda идет аргумент функции (x), двоеточие и после него прописываем действие 
    # над аргументом х

    # В данном случае необходимо из словаря variable_to_label получить значение ключа,
    # а затем по этому значению получить другое значение 
    # из словаря value_labels
    # т.е., value_labels это вложенный словарь - словарь в словаре.
    for name, variable in zip(selection_names[:-1], selection[:-1]):
        print(name)
        if (variable != 'DEN_NA_DUSHU') and not ('(общая площадь)' in name): # доходы и вопросы с общей площадью не по номинальной шкале, так что их преобразовывать не надо
            

            possible_answers = meta.value_labels[meta.variable_to_label[variable]]
            possible_answers_str = {}
            for k, v in possible_answers.items():
                possible_answers_str[str(k)] = v # бывает так, что в этом словаре часть ключей имеет тип данных float, а часть строка
            
            print(f"Возможные ответы: {possible_answers_str}")
            try: 
                variable_values = region[name].apply(lambda x: possible_answers_str[str(x)] if not np.isnan(x) else x)
            except KeyError:
                print("Варианты ответа отсутствуют в словаре возможных ответов. Посчитать осмысленные описательные статистики нельзя")
                print(f"Варианты ответа: {region[name].unique()}")
                print("--------------------------")
                continue
            print(variable_values.describe())
        else:
            print(region[name].describe())
        print("--------------------------")

Что считает метод describe для нечисловых данных:

unique - количество уникальных значений

top - наиболее часто встречающееся значение

freq - частота top-а (мода)

в случае нескольких одинаковых top-ов, какой вывести, выбирается случайно

In [11]:
print("2016 ГОД")
descriptional_statistics(df_2016, meta_2016, selection)

2016 ГОД
Количество наблюдений по выбранным переменным: 1189
Безопасность на улице в темное время суток
Возможные ответы: {'-7.0': 'ЗАТРУДНЯЮСЬ ОТВЕТИТЬ', '1.0': 'Совершенно безопасно', '2.0': 'Достаточно безопасно', '3.0': 'Небезопасно'}
count                      905
unique                       4
top       Достаточно безопасно
freq                       439
Name: Безопасность на улице в темное время суток, dtype: object
--------------------------
Предпочтительное время работы
Возможные ответы: {'-7.0': 'ЗАТРУДНЯЮСЬ ОТВЕТИТЬ', '1.0': 'Полное время работы (по крайней мере, 35 часов в неделю)', '2.0': 'Неполное время работы', '3.0': 'Полное или неполное время работы в определенные месяцы года'}
count                                                   144
unique                                                    3
top       Полное время работы (по крайней мере, 35 часов...
freq                                                    106
Name: Предпочтительное время работы, dtype: object
-----

In [12]:
print("2018 ГОД")
descriptional_statistics(df_2018, meta_2018, selection)

2018 ГОД
Количество наблюдений по выбранным переменным: 1158
Оценка безопасности района проживания респондента в темное время суток
Возможные ответы: {'-7.0': 'Затруднились ответить', '1.0': 'Совершенно безопасно', '2.0': 'Достаточно безопасно', '3.0': 'Небезопасно'}
count                      863
unique                       4
top       Достаточно безопасно
freq                       543
Name: Оценка безопасности района проживания респондента в темное время суток, dtype: object
--------------------------
Предпочтительное время работы
Возможные ответы: {'-7.0': 'Затруднились ответить', '1.0': 'Полное время работы (по крайней мере, 35 часов в неделю)', '2.0': 'Неполное время работы', '3.0': 'Полное или неполное время работы в определенные месяцы года'}
count                                                   132
unique                                                    3
top       Полное время работы (по крайней мере, 35 часов...
freq                                                    10

In [13]:
print("2020 ГОД")
descriptional_statistics(df_2020, meta_2020, selection)


2020 ГОД
Количество наблюдений по выбранным переменным: 1178
Оценка безопасности района проживания респондента в темное время суток
Возможные ответы: {'-7.0': 'Затруднились ответить', '1.0': 'Совершенно безопасно', '2.0': 'Достаточно безопасно', '3.0': 'Небезопасно'}
count                      886
unique                       4
top       Достаточно безопасно
freq                       482
Name: Оценка безопасности района проживания респондента в темное время суток, dtype: object
--------------------------
Предпочтительное время работы
Возможные ответы: {'-7.0': 'Затруднились ответить', '1.0': 'Полное время работы (по крайней мере, 35 часов в неделю)', '2.0': 'Неполное время работы', '3.0': 'Полное или неполное время работы в определенные месяцы года'}
count                                                   116
unique                                                    3
top       Полное время работы (по крайней мере, 35 часов...
freq                                                     8